In [3]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('agg-config.txt')


time_step = int(config.get('Configuration', 'time_step'))
time_step2 = int(config.get('Configuration', 'time_step2'))

window_size = int(config.get('Configuration', 'window_size'))
processed_data_folder = config.get('Configuration', 'processed_data_folder')
raw_data_folder = config.get('Configuration', 'raw_data_folder')

ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')


activities=config.get('Configuration', 'learn_activities')
activities = [(item.strip()) for item in activities.split(',')]

cross_days=config.get('Configuration', 'cross_days')
cross_days = [(item.strip()) for item in cross_days.split(',')]

sensors=config.get('Configuration', 'sensors')
sensors = [(item.strip()) for item in sensors.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))

users_day={}
for cross_day in cross_days:
    users_day[cross_day]=config.get('Configuration', cross_day)
    users_day[cross_day] = [(item.strip()) for item in users_day[cross_day].split(',')]
print(users_day)

{'2024-07-05': ['0001', '3d57', 'ed9c'], '2024-07-06': ['0001', '3d57', 'ed9c'], '2024-07-07': ['0001', '3d57', 'ed9c'], '2024-07-18': ['16fe', 'f02e', 'ed9c'], '2024-07-19': ['16fe', 'f02e', 'ed9c'], '2024-07-20': ['16fe', 'f02e', 'ed9c'], '2024-07-24': ['16fe', '5b66', 'ed9c'], '2024-07-25': ['16fe', '5b66', 'ed9c'], '2024-07-26': ['16fe', '5b66', 'ed9c'], '2024-07-27': ['16fe', '5b66', 'ed9c'], '2024-07-28': ['16fe', '5b66', 'ed9c'], '2024-07-29': ['16fe', '5b66', 'ed9c'], '2024-07-30': ['16fe', '5b66', 'ed9c'], '2024-07-31': ['16fe', '5b66', 'ed9c']}


In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(t0,tN,ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(t0,tN,tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


def day_time0(t0,tN,ti):
    return day_time(ti)-day_time(t0)+1

def relT(t0,tN,ti):
    return (int)((ti-t0)/time_step2)

def sizeT(t0,tN):
    return relT(t0,tN,tN)+1


def initTime(ini_date,end_date):
    tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())

    t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
    print (ini_date,"Init date is", t0, "day:",day_time(t0,tN,t0), time2str(t0,tN,t0))
    print (end_date,"End date is", tN, "day:",day_time(t0,tN,tN), time2str(t0,tN,tN))

    day=list(range(day_time(t0,tN,t0),day_time(t0,tN,tN)+1))[0]
    return day,t0,tN
    
initTime(ini_date,end_date)


def getStrDatefrom(day_number):
    # Reference date (1970-01-01)
    reference_date = datetime(1970, 1, 1)
    
    # Compute the date corresponding to the day number
    resulting_date = reference_date + timedelta(days=day_number)
    
    return resulting_date.strftime("%Y-%m-%d")


2024-07-31 02:00:00 Init date is 1722384000 day: 19935 2024-07-31 02:00:00
2024-08-01 01:59:59 End date is 1722470399 day: 19936 2024-08-01 01:59:59


In [3]:
from datetime import datetime, timedelta

days=[]
t0N_days={}

for cross_day in cross_days:
    print(cross_day)
    # Convertir las fechas a objetos datetime
    ini_date_dt = datetime.strptime(ini_date, "%Y-%m-%d %H:%M:%S")
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")
    cross_day_dt = datetime.strptime(cross_day, "%Y-%m-%d")

    # Reemplazar el día manteniendo la hora original
    new_ini_date = cross_day_dt.replace(hour=ini_date_dt.hour, minute=ini_date_dt.minute, second=ini_date_dt.second)
    new_end_date = cross_day_dt.replace(hour=end_date_dt.hour, minute=end_date_dt.minute, second=end_date_dt.second) + timedelta(days=1)

    # Convertir las fechas de nuevo a cadena si es necesario
    new_ini_date_str = new_ini_date.strftime("%Y-%m-%d %H:%M:%S")
    new_end_date_str = new_end_date.strftime("%Y-%m-%d %H:%M:%S")

    print(f"New ini_date: {new_ini_date_str}")
    print(f"New end_date: {new_end_date_str}")
    day,t0,tN=initTime(new_ini_date_str,new_end_date_str)
    days.append(day)
    t0N_days[day]=(t0,tN)
    

print(days)
print(t0N_days)



2024-07-05
New ini_date: 2024-07-05 02:00:00
New end_date: 2024-07-06 01:59:59
2024-07-05 02:00:00 Init date is 1720137600 day: 19909 2024-07-05 02:00:00
2024-07-06 01:59:59 End date is 1720223999 day: 19910 2024-07-06 01:59:59
2024-07-06
New ini_date: 2024-07-06 02:00:00
New end_date: 2024-07-07 01:59:59
2024-07-06 02:00:00 Init date is 1720224000 day: 19910 2024-07-06 02:00:00
2024-07-07 01:59:59 End date is 1720310399 day: 19911 2024-07-07 01:59:59
2024-07-07
New ini_date: 2024-07-07 02:00:00
New end_date: 2024-07-08 01:59:59
2024-07-07 02:00:00 Init date is 1720310400 day: 19911 2024-07-07 02:00:00
2024-07-08 01:59:59 End date is 1720396799 day: 19912 2024-07-08 01:59:59
2024-07-18
New ini_date: 2024-07-18 02:00:00
New end_date: 2024-07-19 01:59:59
2024-07-18 02:00:00 Init date is 1721260800 day: 19922 2024-07-18 02:00:00
2024-07-19 01:59:59 End date is 1721347199 day: 19923 2024-07-19 01:59:59
2024-07-19
New ini_date: 2024-07-19 02:00:00
New end_date: 2024-07-20 01:59:59
2024-07-1

In [4]:
import os
import pandas as pd
len_users=3
loc_user=np.zeros((len(activities),len_users,len(days),sizeT(t0,tN)))
print(loc_user.shape)
for ax,activity in enumerate(activities):
    print("\t",activity)
    for dx,day in enumerate(days):
        day_str=getStrDatefrom(day)
        users=users_day[day_str]
        print(day,users)
        for ux,user in enumerate(users):
            print(processed_data_folder+"/DAY_"+str(day)+"/act/loc."+user+"."+activity+".tsv")
            series=read_csv(processed_data_folder+"/DAY_"+str(day)+"/act/loc."+user+"."+activity+".tsv", sep="\t",parse_dates=True, header=None, names=["time","loc"])
            series=series["loc"].values
            print(day,series.shape)
            loc_user[ax][ux][dx]=series
    


(8, 3, 14, 1440)
	 toileting
19909 ['0001', '3d57', 'ed9c']
./data-har//DAY_19909/act/loc.0001.toileting.tsv
19909 (1440,)
./data-har//DAY_19909/act/loc.3d57.toileting.tsv
19909 (1440,)
./data-har//DAY_19909/act/loc.ed9c.toileting.tsv
19909 (1440,)
19910 ['0001', '3d57', 'ed9c']
./data-har//DAY_19910/act/loc.0001.toileting.tsv
19910 (1440,)
./data-har//DAY_19910/act/loc.3d57.toileting.tsv
19910 (1440,)
./data-har//DAY_19910/act/loc.ed9c.toileting.tsv
19910 (1440,)
19911 ['0001', '3d57', 'ed9c']
./data-har//DAY_19911/act/loc.0001.toileting.tsv
19911 (1440,)
./data-har//DAY_19911/act/loc.3d57.toileting.tsv
19911 (1440,)
./data-har//DAY_19911/act/loc.ed9c.toileting.tsv
19911 (1440,)
19922 ['16fe', 'f02e', 'ed9c']
./data-har//DAY_19922/act/loc.16fe.toileting.tsv
19922 (1440,)
./data-har//DAY_19922/act/loc.f02e.toileting.tsv
19922 (1440,)
./data-har//DAY_19922/act/loc.ed9c.toileting.tsv
19922 (1440,)
19923 ['16fe', 'f02e', 'ed9c']
./data-har//DAY_19923/act/loc.16fe.toileting.tsv
19923 (1440

In [5]:
act_sensors=np.zeros((len(sensors),len(days),sizeT(t0,tN)))
print(act_sensors.shape)
for sx,sensor in enumerate(sensors):
    print("\t",sensor)
    for dx,day in enumerate(days):
        print(processed_data_folder+"/DAY_"+str(day)+"/object_sensor/"+sensor+".tsv")
        series=read_csv(processed_data_folder+"/DAY_"+str(day)+"/object_sensor/"+sensor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","status"])
        series=series["status"].values
        print(day,series.shape)
        act_sensors[sx][dx]=series

(11, 14, 1440)
	 fridge_13
./data-har//DAY_19909/object_sensor/fridge_13.tsv
19909 (1440,)
./data-har//DAY_19910/object_sensor/fridge_13.tsv
19910 (1440,)
./data-har//DAY_19911/object_sensor/fridge_13.tsv
19911 (1440,)
./data-har//DAY_19922/object_sensor/fridge_13.tsv
19922 (1440,)
./data-har//DAY_19923/object_sensor/fridge_13.tsv
19923 (1440,)
./data-har//DAY_19924/object_sensor/fridge_13.tsv
19924 (1440,)
./data-har//DAY_19928/object_sensor/fridge_13.tsv
19928 (1440,)
./data-har//DAY_19929/object_sensor/fridge_13.tsv
19929 (1440,)
./data-har//DAY_19930/object_sensor/fridge_13.tsv
19930 (1440,)
./data-har//DAY_19931/object_sensor/fridge_13.tsv
19931 (1440,)
./data-har//DAY_19932/object_sensor/fridge_13.tsv
19932 (1440,)
./data-har//DAY_19933/object_sensor/fridge_13.tsv
19933 (1440,)
./data-har//DAY_19934/object_sensor/fridge_13.tsv
19934 (1440,)
./data-har//DAY_19935/object_sensor/fridge_13.tsv
19935 (1440,)
	 tap_22
./data-har//DAY_19909/object_sensor/tap_22.tsv
19909 (1440,)
./data-

In [6]:
user_activity=np.zeros((len(activities),len(users),len(days),sizeT(t0,tN)))
for dx,day in enumerate(days):
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    (t0,tN)=t0N_days[day]
    ts=list(range(t0,tN,time_step2))
    for ax,activity in enumerate(activities):
        for ux,user in enumerate(users):
            df_total=read_csv(processed_data_folder+"/DAY_"+str(day)+"/act/INTERVAL."+user+"."+activity+".tsv", sep="\t",parse_dates=True, header=None, names=["ix","d0","dN"])
            #print(df_total)
            #print(pd.to_datetime(df_total['d0']))
            df_total['d0'] = pd.to_datetime(df_total['d0'])
            df_total['dN'] = pd.to_datetime(df_total['dN'])
            #print(df_total)

            for tx,tt in enumerate(ts):
                ttt=pd.to_datetime(time2str(t0,tN,tt))
#                print(ttt)
                row = df_total[(df_total['d0'] <= ttt) & (df_total['dN'] >= ttt)]
#                print("df_in#tervalo",row)
                if(row.size>0):
                    user_activity[ax][ux][dx][tx]=1

for dx,day in enumerate(days):
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    (t0,tN)=t0N_days[day]
    ts=list(range(t0,tN,time_step2))
            
    for ax,activity in enumerate(activities):
        for ux,user in enumerate(users):
            with open(processed_data_folder+"/DAY_"+str(day)+"/act/LABEL3."+user+"."+activity+".tsv", "w") as file:
                for tx,tt in enumerate(ts):
                    file.write(time2str(t0,tN,tt)+"\t"+str(user_activity[ax][ux][dx][tx])+"\n")




19909 ['0001', '3d57', 'ed9c']
19910 ['0001', '3d57', 'ed9c']
19911 ['0001', '3d57', 'ed9c']
19922 ['16fe', 'f02e', 'ed9c']
19923 ['16fe', 'f02e', 'ed9c']
19924 ['16fe', 'f02e', 'ed9c']
19928 ['16fe', '5b66', 'ed9c']
19929 ['16fe', '5b66', 'ed9c']
19930 ['16fe', '5b66', 'ed9c']
19931 ['16fe', '5b66', 'ed9c']
19932 ['16fe', '5b66', 'ed9c']
19933 ['16fe', '5b66', 'ed9c']
19934 ['16fe', '5b66', 'ed9c']
19935 ['16fe', '5b66', 'ed9c']
19909 ['0001', '3d57', 'ed9c']
19910 ['0001', '3d57', 'ed9c']
19911 ['0001', '3d57', 'ed9c']
19922 ['16fe', 'f02e', 'ed9c']
19923 ['16fe', 'f02e', 'ed9c']
19924 ['16fe', 'f02e', 'ed9c']
19928 ['16fe', '5b66', 'ed9c']
19929 ['16fe', '5b66', 'ed9c']
19930 ['16fe', '5b66', 'ed9c']
19931 ['16fe', '5b66', 'ed9c']
19932 ['16fe', '5b66', 'ed9c']
19933 ['16fe', '5b66', 'ed9c']
19934 ['16fe', '5b66', 'ed9c']
19935 ['16fe', '5b66', 'ed9c']


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, concatenate, GRU, TimeDistributed, Attention, Add, Dropout,Conv1D
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, Conv1D, TimeDistributed, Flatten, Dropout, GRU, Attention, Add, Dense, Concatenate
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, Concatenate, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention


N1 = len(activities)
N2 = len(activities)
N3 = len(sensors)

TWA = 30
TWB = 15
T = TWA + TWB

def getModelHAR():
    # Define inputs
    input1 = Input(shape=(N1, T, 1))
    input3 = Input(shape=(N3, T, 1))

    # Reshape inputs for the Transformer
    reshaped1 = Reshape((N1, T))(input1)
    reshaped3 = Reshape((N3, T))(input3)

    # Transformer block for the first sequence
    transformer1 = MultiHeadAttention(key_dim=64, num_heads=4)(reshaped1, reshaped1)
    transformer1 = GlobalAveragePooling1D()(transformer1)

    # Transformer block for the third sequence
    transformer3 = MultiHeadAttention(key_dim=64, num_heads=4)(reshaped3, reshaped3)
    transformer3 = GlobalAveragePooling1D()(transformer3)

    # Concatenation of the three Transformer blocks
    merged = Concatenate()([transformer1,transformer3])
    merged = Dropout(0.5)(merged)

    # Dense layer for binary classification
    dense = Dense(128, activation='relu')(merged)
    dense = Dropout(0.5)(dense)
    dense = Dense(32, activation='relu')(dense)
    output = Dense(1, activation='sigmoid')(dense)

    # Define the model
    model = Model(inputs=[input1, input3], outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    return model




# Instanciación del modelo
model = getModelHAR()
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 8, 45, 1)]   0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 11, 45, 1)]  0           []                               
                                                                                                  
 reshape_2 (Reshape)            (None, 8, 45)        0           ['input_3[0][0]']                
                                                                                                  
 reshape_3 (Reshape)            (None, 11, 45)       0           ['input_4[0][0]']                
                                                                                            

In [ ]:
import random



SizeTrainOk=4000
SizeTrainKo=6000


for dx,day in enumerate(days):
    print("\t",dx,day)
    other_dx= [other for other in range(0,len(days)) if other != dx]

    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    
    for ax,activity in enumerate(activities):
            print(ax,activity)
            
            ##TRAING DATA
            L1=[]
            L2=[]
            S=[]
            Y=[]
            for ux,user in enumerate(users):
                    print("\t\t",ux,user)
                    other_ux= [other for other in range(0,len(users)) if other != ux]
                    print(day,user,other_dx,other_ux)

                    #OK act
                    for ix in range(0,SizeTrainOk):
                        #print("SizeTrainOk",ix,"vs",SizeTrainOk)
                        filtered_elements = user_activity[ax, ux, other_dx, :]

                        # Encontramos los elementos que son 1
                        indices = np.argwhere(filtered_elements == 1)

                        if(len(indices)==0):
                            #print("\t\t\t","Non activity for user ")
                            break
                        
                        # Para obtener las posiciones relativas a las dimensiones originales
                        indexs = [(ax, ux, other_dx[idx[0]], idx[1]) for idx in indices]
                        index = random.choice(indexs)
                        #print("\t\t\t",index,user_activity[index])
                        
                        (ia,iu,idy,tt)=index
                        
                        if(tt<TWA):
                            tt=TWA
                        if(tt>=sizeT(t0,tN)-TWB):
                            tt=sizeT(t0,tN)-TWB
                        
                        
                        X1s=[]
                        X2s=[]
                        for ax2,activity2 in enumerate(activities):
                            X1s.append(loc_user[ax2][iu][idy][tt-TWA:tt+TWB])
                            X2s.append(np.amax(loc_user[ax2,other_ux,idy,tt-TWA:tt+TWB],axis=0))
                            
                            
                        Ss=[]
                        for sx, sensor in enumerate(sensors):
                            Ss.append(act_sensors[sx][idy][tt-TWA:tt+TWB])
                        L1.append(X1s)
                        L2.append(X2s)
                        S.append(Ss)
                        Y.append(user_activity[index])
                        
                    #KO act
                    ix=0
                    while ix < SizeTrainKo:
                        #print("SizeTrainKo",ix,"vs",SizeTrainKo)
                        filtered_elements_specific = user_activity[ax, ux, other_dx, :]

                        # Encontramos los elementos que son 1
                        indices = np.argwhere(filtered_elements == 0)

                        if(len(indices)==0):
                            print("\t\t\t","Non activity for user ")
                            break
                        
                        # Para obtener las posiciones relativas a las dimensiones originales
                        indexs = [(ax, ux, other_dx[idx[0]], idx[1]) for idx in indices]
                        index = random.choice(indexs)
                        #print("\t\t\t",index,user_activity[index])
                        
                        (ia,iu,idy,tt)=index
                        
                        if(tt<TWA):
                            tt=TWA
                        if(tt>=sizeT(t0,tN)-TWB):
                            tt=sizeT(t0,tN)-TWB
                        
                        
                        X1s=[]
                        X2s=[]
                        none=True
                        for ax2,activity2 in enumerate(activities):
                            #if(ax2==ax):
                                r=np.random.rand()
                                #print(" removing loc?:",loc_user[ax2][iu][idy][tt],r)
                                if(r<loc_user[ax2][iu][idy][tt]+0.05):
                                    #print("NOT  removing loc:",loc_user[ax2][iu][idy][tt])
                                    none=False
                                    
                                X1s.append(loc_user[ax2][iu][idy][tt-TWA:tt+TWB])
                                X2s.append(np.amax(loc_user[ax2,other_ux,idy,tt-TWA:tt+TWB],axis=0))
                            
                            
                        if not none:
                            L1.append(X1s)
                            L2.append(X2s)
                            Ss=[]
                            for sx, sensor in enumerate(sensors):
                                Ss.append(act_sensors[sx][idy][tt-TWA:tt+TWB])
                            S.append(Ss)
                            Y.append(user_activity[index])       
                            ix=ix+1

            L1=np.array(L1)        
            L2=np.array(L2)        
            S=np.array(S)
            Y=np.array(Y)
            print("TRAIN",activity,day,L1.shape,L2.shape,S.shape,Y.shape)
            
            
            model=getModelHAR()
            model.summary()
            
            ##TEST DATA
            TL1=[]
            TL2=[]
            TS=[]
            TY=[]
            
            
            (t0,tN)=t0N_days[day]
            ts=list(range(TWA,sizeT(t0,tN)-TWB))
            
            for ux2,user2 in enumerate(users):
                other_ux2= [other for other in range(0,len(users)) if other != ux2]

                for tx in ts:

                    Ss=[]
                    for sx, sensor in enumerate(sensors):
                        Ss.append(act_sensors[sx][dx][tx-TWA:tx+TWB])
                    
                    X1s=[]
                    X2s=[]
                    for ax2,activity2 in enumerate(activities):
                        X1s.append(loc_user[ax2][ux2][dx][tx-TWA:tx+TWB])
                        X2s.append(np.amax(loc_user[ax2,other_ux2,dx,tx-TWA:tx+TWB],axis=0))


                    TL1.append(X1s)
                    TL2.append(X2s)
                    TS.append(Ss)
                    TY.append(user_activity[ax, ux2, dx, tx])
                        
            TL1=np.array(TL1)        
            TL2=np.array(TL2)        
            TS=np.array(TS)
            TY=np.array(TY)
            print("EVAL",activity,day,TL1.shape,TL2.shape,TS.shape,TY.shape)
            
            history = model.fit([L1,S] , Y, shuffle=True, batch_size=32,epochs=10, validation_data = ([TL1,TS], TY))
            YT_prediction = model.predict([TL1,TS])
            print(YT_prediction.shape)
            
            ix=0
            for ux2,user2 in enumerate(users):
                with open(processed_data_folder+"/DAY_"+str(day)+"/act/PREDICTION-TRANS1-other-30+15."+user2+"."+activity+".tsv", "w") as file:
                    for tx in (ts):
                        tt=tx*time_step2+t0
                        #print("\t"+time2str(t0,tN,tt)+"\t"+str(TY[ix])+"\t"+str(YT_prediction[ix][0]))
                        file.write("\t"+time2str(t0,tN,tt)+"\t"+str(TY[ix])+"\t"+str(YT_prediction[ix][0])+"\n")
                        
                        ix=ix+1




	 0 19909
19909 ['0001', '3d57', 'ed9c']
0 toileting
		 0 0001
19909 0001 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [1, 2]
		 1 3d57
19909 3d57 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [0, 2]
		 2 ed9c
19909 ed9c [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [0, 1]
TRAIN toileting 19909 (30000, 8, 45) (30000, 8, 45) (30000, 11, 45) (30000,)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 8, 45, 1)]   0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 11, 45, 1)]  0           []                               
                                                                                                  
 reshape_2 (Reshape)            (None, 8, 4

2024-11-30 20:41:52.870475: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-11-30 20:41:52.989983: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f4896e57230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-30 20:41:52.990020: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-11-30 20:41:53.013104: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-30 20:41:53.069913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-11-30 20:41:53.283051: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

938/938 [==============================] - 12s 9ms/step - loss: 0.2139 - accuracy: 0.9190 - val_loss: 0.1113 - val_accuracy: 0.9551
Epoch 2/10
938/938 [==============================] - 8s 9ms/step - loss: 0.1342 - accuracy: 0.9531 - val_loss: 0.1853 - val_accuracy: 0.9326
Epoch 3/10
938/938 [==============================] - 8s 8ms/step - loss: 0.1164 - accuracy: 0.9604 - val_loss: 0.1078 - val_accuracy: 0.9536
Epoch 4/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0995 - accuracy: 0.9672 - val_loss: 0.1142 - val_accuracy: 0.9536
Epoch 5/10
938/938 [==============================] - 8s 8ms/step - loss: 0.0888 - accuracy: 0.9713 - val_loss: 0.1285 - val_accuracy: 0.9472
Epoch 6/10
938/938 [==============================] - 8s 8ms/step - loss: 0.0784 - accuracy: 0.9754 - val_loss: 0.0882 - val_accuracy: 0.9704
Epoch 7/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0704 - accuracy: 0.9772 - val_loss: 0.1786 - val_accuracy: 0.9427
Epoch 8/10
938/9

 gru_22 (GRU)                (None, 11, 128)              2261760   ['time_distributed_44[0][0]'] 
                                                                                                  
 concatenate_4 (Concatenate  (None, 27, 128)              0         ['gru_20[0][0]',              
 )                                                                   'gru_21[0][0]',              
                                                                     'gru_22[0][0]']              
                                                                                                  
 dropout_20 (Dropout)        (None, 27, 128)              0         ['concatenate_4[0][0]']       
                                                                                                  
 gru_23 (GRU)                (None, 27, 256)              296448    ['dropout_20[0][0]']          
                                                                                                  
 dropout_2

 time_distributed_47 (TimeD  (None, 8, 5760)              0         ['time_distributed_46[0][0]'] 
 istributed)                                                                                      
                                                                                                  
 time_distributed_50 (TimeD  (None, 8, 5760)              0         ['time_distributed_49[0][0]'] 
 istributed)                                                                                      
                                                                                                  
 time_distributed_53 (TimeD  (None, 11, 5760)             0         ['time_distributed_52[0][0]'] 
 istributed)                                                                                      
                                                                                                  
 gru_25 (GRU)                (None, 8, 128)               2261760   ['time_distributed_47[0][0]'] 
          